# 手写 word2vec 模型和训练

word2vec 不是一种单一算法，而是一系列模型架构和优化，可用于从大型数据集中学习单词嵌入。通过 word2vec 学习的嵌入已被证明在各种下游自然语言处理任务上是成功的。

Note: 本教程基于 [Efficient estimation of word representations in vector space](https://arxiv.org/pdf/1301.3781.pdf) 和 [Distributed representations of words and phrases and their compositionality](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)。 这不是论文的具体实施。相反，它旨在说明关键思想。

这些论文提出了两种学习单词表示的方法：

*   **Continuous bag-of-words model**: 基于周围上下文词预测中间词。上下文由当前（中间）单词前后的几个单词组成。这种体系结构被称为单词包模型，因为上下文中单词的顺序并不重要。

*   **Continuous skip-gram model**: 预测同一句子中当前单词前后一定范围内的单词。下面给出了一个工作示例。

在本教程中，您将使用 skip-gram 方法。首先，您将使用一个句子来说明跳过语法和其他概念。接下来，您将在一个小数据集上训练自己的word2vec模型。本教程还包含导出经过训练的嵌入并在中可视化它们的代码 [TensorFlow Embedding Projector](http://projector.tensorflow.org/).


## Skip-gram 和负采样 (negative sampling)

当 bag-of-words 模型预测给定相邻上下文的单词时，skip-gram 模型预测给定单词本身的单词上下文（或相邻）。该模型基于跳过图进行训练，skip-gram 是允许跳过 tokens 的 n-grams（参见下图）。一个词的上下文可以通过一组 “(target_word，context_words)” 的跳过语法对来表示，其中 “context_ word” 出现在 “target-word” 的相邻上下文中。

考虑以下八个单词的句子：

> The wide road shimmered in the hot sun.

该句子的 8 个单词中的每一个的上下文单词由窗口大小定义。窗口大小确定可被视为“上下文词”的 “target_word” 两侧的词的跨度。下面是基于不同窗口大小的目标词的 skip-grams。

Note: 对于本教程，窗口大小 “n” 表示每侧有 n 个单词，整个窗口跨度为 2*n+1 个单词。

![word2vec_skipgrams](https://aimaksen.rarelimiting.com/course/word2vec_skipgram.png)

skip-gram 模型的训练目标是最大化预测给定目标词的上下文词的概率。对于一系列单词 *w<sub>1</sub>、w<sub>2</sub>、…w<sub>T</sub>*，目标可以写成平均对数概率。

![word2vec_skipgram_objective](https://aimaksen.rarelimiting.com/course/word2vec_skipgram_objective.png)

其中 ‘c’ 是训练上下文的大小。基本跳跃图公式使用 softmax 函数定义该概率。

![word2vec_full_softmax](https://aimaksen.rarelimiting.com/course/word2vec_full_softmax.png)

其中，*v* 和 *v<sup>'<sup>* 是单词的目标和上下文向量表示，*W* 是词汇大小。

计算该公式的分母涉及对整个词汇表单词执行完整的 softmax，这些单词通常是大的（10<sup>5</sup>-10<supp>7</supp>）。

[noise contrastive estimation](https://www.tensorflow.org/api_docs/python/tf/nn/nce_loss) (NCE) 损失函数是完整softmax的有效近似。为了学习单词嵌入而不是建模单词分布，NCE 损失可以使用负采样方法 [simplified](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) 

目标词的简化负采样目标是将上下文词与从词的噪声分布 *P<sub>n</sub>（w）* 中提取的 “num_ns” 负样本区分开来。更准确地说，词汇表上的完整 softmax 的有效近似是，对于跳转语法对，将目标词的丢失作为上下文词和 “num_ns” 负样本之间的分类问题。

负样本被定义为 `（target_word，context_words）` 对，使得 ` context_ word'不出现在` target_ word'的 ` window_size` 邻域中。对于示例语句，这是几个潜在的负样本（当 ‘window_size’ 为 ‘2’ 时）。

```
(hot, shimmered)
(wide, hot)
(wide, sun)
```

在下一节中，您将为单个句子生成 skip-gram 和负样本。在本教程的后面部分，您还将学习子采样技术，并为正和负训练示例训练分类模型。

## 安装包

In [4]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

2024-01-22 18:34:24.452695: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

### 矢量化例句

考虑以下句子：

> The wide road shimmered in the hot sun.

给句子分词:

In [4]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

8


创建词汇表以保存从令牌到整数索引的映射：

In [5]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


创建反向词汇表以保存从整数索引到令牌的映射：

In [6]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


矢量化你的句子：

In [7]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


### 从一个句子生成 skip-grams 

`tf.keras.preprocessing` “序列”模块提供了有用的功能，简化了 word2vec 的数据准备。您可以使用 `tf.keras.preprocessing.sequence`。skip-grams 用于从具有给定 ‘window_size’ 的 ‘example_sequence’ 中从范围 `[0，vocab_size）` 中的标记生成跳过图对。

Note: 在这里，`negative_samples` 设置为 0，因为批处理此函数生成的负样本需要单独的代码。在下一节中，您将使用另一个函数执行负采样。

In [8]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))

26


打印几个 skip-grams：

In [9]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(7, 1): (sun, the)
(3, 1): (road, the)
(5, 6): (in, hot)
(5, 1): (in, the)
(6, 7): (hot, sun)


### skip-gram 的负采样

`skipgrams` 函数通过在给定的窗口跨度上滑动来返回所有正的 skip-grams 对。要生成额外的跳过语法对，用作训练的负样本，您需要从词汇表中随机抽取单词。使用 `tf.random`。log_uniform_candidate_sampler 函数对窗口中给定目标字的负样本数进行采样。您可以在一个 skip-grams 的目标词上调用该函数，并将上下文词作为 true 类传递，以将其从采样中排除。

关键点: 对于小的数据集 `num_ns` (the number of negative samples per a positive context word) 的范围在 `[5, 20]`  [shown to work](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) ， 对于的大的数据集 `num_ns` 的范围在 `[2, 5]`。

In [10]:
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['wide', 'the', 'shimmered', 'road']


### 构建一个训练样本示例 

For a given positive `(target_word, context_word)` skip-gram, you now also have `num_ns` negative sampled context words that do not appear in the window size neighborhood of `target_word`. Batch the `1` positive `context_word` and `num_ns` negative context words into one tensor. This produces a set of positive skip-grams (labeled as `1`) and negative samples (labeled as `0`) for each target word

对于给定的正 `（target_word，context_words）` skip-gram，您现在也有 `num_ns` 负采样的上下文字，它们不出现在 `target_ word` 的窗口大小附近。将 1 设置为正 context_word 和 num_ns 负上下文词批处理为一个张量。这为每个目标字产生一组正跳转图（标记为“1”）和负样本（标记为”0”）。

In [11]:
# Add a dimension so you can use concatenation (in the next step).
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")

# Reshape the target to shape `(1,)` and context and label to `(num_ns+1,)`.
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

从上面的 skip-gram 示例中查看目标词的上下文和相应标签：

In [12]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 7
target_word     : sun
context_indices : [1 2 1 4 3]
context_words   : ['the', 'wide', 'the', 'shimmered', 'road']
label           : [1 0 0 0 0]


一个由`（target，context，label）` 张量组成的元组构成了一个训练示例，用于训练跳过革兰氏阴性采样word2vec模型。请注意，目标的形状为 `（1，）`，而上下文和标签的 shape 为 `（1+num_ns，）`

In [13]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : tf.Tensor(7, shape=(), dtype=int32)
context : tf.Tensor([1 2 1 4 3], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


### 总结

此图总结了从句子生成训练示例的过程：

![image.png](https://aimaksen.rarelimiting.com/course/1705801821737.jpg)

注意，`temperature` 和 `code` 不是输入句子的一部分。它们与上图中使用的某些其他索引一样属于词汇表。

## 将所有步骤融合到一个函数


### Skip-gram 采样表

大数据集意味着词汇表更大，更频繁的词（如停止词）数量更多。通过对常见单词（如“the”、“is”、“on”）进行采样而获得的训练示例并没有为模型提供多少有用的信息。[Mikolov et al.](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) 建议对频繁词进行二次采样，以提高嵌入质量。

`tf.keras.preprocessing.sequence.skipgrams` 函数接受一个采样表参数，以编码对任何令牌进行采样的概率。您可以使用`tf.keras.preprocessing.sequence`。make_sampling_table 生成基于词频秩的概率采样表，并将其传递给 skipgrams 函数。检查 vocab_size 为 10 的采样概率。

In [14]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


`sampling_table[i]` 表示对数据集中第i个最常用单词进行采样的概率。该函数基于字频率采样。 [Zipf's distribution](https://en.wikipedia.org/wiki/Zipf%27s_law)

关键点: `tf.random.log_uniform_candidate_sampler` 已经假设词汇频率遵循对数均匀（Zipf）分布。使用这些分布加权采样也有助于使用更简单的损失函数来近似噪声对比估计（NCE）损失，以训练负采样目标。

### 生成训练数据

将上述所有步骤编译成一个函数，该函数可以在从任何文本数据集获得的矢量化语句列表上调用。请注意，采样表是在对跳过字对进行采样之前构建的。您将在后面的章节中使用此函数。

In [15]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

## 为 word2vec 准备训练数据

了解了如何使用一个基于 skip-gram 负采样的 word2vec 模型的句子，您可以继续从更大的句子列表中生成训练示例！

### 下载文本语料库

本教程将使用莎士比亚作品的文本文件。更改以下行以在您自己的数据上运行此代码。

In [5]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

从文件中读取文本并打印前几行：

In [17]:
with open(path_to_file) as f:
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


使用非空行构造`tf.data.TextLineDataset`对象，用于下一步：

In [18]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

### 从语料库中矢量化句子

你可以使用 `TextVectorization` 把 corpus 中的句子向量化。 关于 `TextVectorization` 可以从 [Text classification](https://www.tensorflow.org/tutorials/keras/text_classification) 得到更多的细节. 请注意，从上面的前几句话中，文本需要在一种情况下，标点符号需要删除。为了做到这个，我们需要定义 `custom_standardization` 函数， 可以在文本矢量化层中使用。

In [19]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

调用 `TextVectorization.adapt` 在文本数据集上使用 “adapt ”创建词汇表。

In [20]:
vectorize_layer.adapt(text_ds.batch(1024))

一旦该层的状态被调整为表示文本语料库，就可以使用 `TextVectorization.get_vocabulary` 获取词典。 此函数返回按频率排序（降序）的所有词汇表标记的列表。

In [21]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


`vectorize_layer` 可以对于每一个元素生成向量 `text_ds` (`tf.data.Dataset`)。并可以应用 `Dataset.batch`, `Dataset.prefetch`, `Dataset.map` 和 `Dataset.unbatch`.

In [22]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

### 从数据集中获取序列


您现在有了一个 `tf.data.Dataset`。整数编码句子的数据集。要准备用于训练 word2vec 模型的数据集，请将数据集展平为句子向量序列列表。这一步是必需的，因为您将迭代数据集中的每个句子，以生成正负样本。

Note:  `generate_training_data()` 在前面定义的使用非TensorFlow Python/NumPy函数时，您还可以使用 `tf.py_function` 或 `tf.numpy_function` 和 `tf.data.Dataset.map`.

In [23]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

32777


查看以下示例 `sequences`:

In [24]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


### 从序列生成训练示例

`sequences` 现在是int编码语句的列表。只需调用前面定义的 `generate_training_data` 函数即可为 word2vec 模型生成训练示例。总而言之，该函数迭代每个序列中的每个单词，以收集正面和负面上下文单词。目标、上下文和标签的长度应相同，表示训练示例的总数。

In [25]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32777/32777 [00:16<00:00, 2023.74it/s]




targets.shape: (65287,)
contexts.shape: (65287, 5)
labels.shape: (65287, 5)


### 为性能配置数据集

要对可能大量的训练示例执行有效的批处理，请使用 `tf.data.Dataset` API. 在这一步之后， `tf.data.Dataset` 中包含 `(target_word, context_word), (label)`  元素去训练 word2vec 模型!

In [26]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<_BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


应用 `Dataset.cache` 和 `Dataset.prefetch` 去提高性能:

In [27]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


## 模型和训练

word2vec 模型可以被实现为分类器，以区分来自跳过图的真实上下文词和通过负采样获得的虚假上下文词。您可以在目标词和上下文词的嵌入之间执行点积乘法，以获得标签预测，并根据数据集中的真实标签计算损失函数。

### word2vec 模型类

使用 [Keras Subclassing API](https://www.tensorflow.org/guide/keras/custom_layers_and_models) 去定义你的 to define your word2vec 模型:

* `target_embedding`: `tf.keras.layers.Embedding` 层, 我们可以从中去查找目标单词的 embedding，这一层的超参数量为 `(vocab_size * embedding_dim)`

* `context_embedding`: 另外 `tf.keras.layers.Embedding` 层, 当一个词作为上下文词出现时，查找该词的嵌入。 该层中的参数数量与 `target_embedding`, 比如 `(vocab_size * embedding_dim)` 中的相同

* `dots`: `tf.keras.layers.Dot` 从训练对计算目标和上下文嵌入的点积的层。

* `flatten`: `tf.keras.layers.Flatten` layer 将 `dots` 层的结果展平为 Logit。 

使用子类模型，您可以定义接受 `（target，context）` 对的 ` call（） `函数，然后将其传递到相应的嵌入层。重塑 “context_embedded” 以执行与“target_embodded” 的点积，并返回展平后的结果。

关键点: “target_embedded” 和 “context_embudded” 层也可以共享。您还可以使用两个嵌入的串联作为最终的 word2vec 嵌入。

In [28]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

### 定义损失函数并编译模型


为了简单起见，您可以使用 `tf.keras.loss.CategoricalCrossEntropy` 作为负采样损失的替代方案。如果您想编写自己的自定义损耗函数，也可以按如下方式编写：

``` python
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)
```

是时候建立你的模型了！使用128的嵌入维度实例化word2vec类（您可以使用不同的值进行实验）。使用 `tf.keras.optimizers.Adam` 优化器编译模型。

In [29]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

还定义一个回调来记录 TensorBoard 的训练统计信息：

In [30]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

在 `dataset` 上对模型进行若干 epochs 的训练：

In [31]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
63/63 [==============================] - 1s 10ms/step - loss: 1.6083 - accuracy: 0.2327
Epoch 2/20
63/63 [==============================] - 1s 9ms/step - loss: 1.5892 - accuracy: 0.5538
Epoch 3/20
63/63 [==============================] - 1s 9ms/step - loss: 1.5420 - accuracy: 0.6037
Epoch 4/20
63/63 [==============================] - 1s 10ms/step - loss: 1.4592 - accuracy: 0.5798
Epoch 5/20
63/63 [==============================] - 1s 10ms/step - loss: 1.3610 - accuracy: 0.5836
Epoch 6/20
63/63 [==============================] - 1s 11ms/step - loss: 1.2643 - accuracy: 0.6088
Epoch 7/20
63/63 [==============================] - 1s 10ms/step - loss: 1.1744 - accuracy: 0.6422
Epoch 8/20
63/63 [==============================] - 1s 10ms/step - loss: 1.0909 - accuracy: 0.6753
Epoch 9/20
63/63 [==============================] - 1s 10ms/step - loss: 1.0131 - accuracy: 0.7078
Epoch 10/20
63/63 [==============================] - 1s 9ms/step - loss: 0.9406 - accuracy: 0.7374
Epoch 11/20


TensorBoard 现在显示 word2vec 模型的精度和损失：

In [32]:
#docs_infra: no_execute
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="images/word2vec_tensorboard.png"/> -->

## 词嵌入查找和分析

Obtain the weights from the model using `Model.get_layer` and `Layer.get_weights`. The `TextVectorization.get_vocabulary` function provides the vocabulary to build a metadata file with one token per line.

使用 `Layer.get_weights` 和 `Model.get_layer` 从模型中获取权重。`TextVectorization.get_vocabulary` 函数提供词汇表的获取。

In [33]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

创建并保存矢量和元数据文件：

In [34]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

下载 `vectors.tsv` 和 `metadata.tsv` 去在 [Embedding Projector](https://projector.tensorflow.org/) 中可视化分析:

In [35]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

# 开始在Gensim中使用Word2Vec并使其发挥作用！

Word2Vec背后的想法非常简单。我们假设，你可以通过一个词的公司来判断它的含义。这类似于“给我看看你的朋友，我就会告诉你是谁”这句话。因此，如果你有两个单词有非常相似的邻居（即使用上下文大致相同），那么这些单词的含义可能非常相似，或者至少高度相关。例如，“震惊”、“震惊”和“惊讶”等词通常用于类似的上下文。

在本教程中，您将学习如何使用Word2Vec的Gensim实现，并使其真正发挥作用！我听过很多关于性能差等的抱怨，但这实际上是两件事的结合，（1）你的输入数据和（2）你的参数设置。请注意，此包中的训练算法是从[谷歌最初的Word2Vec实现]移植而来的(https://arxiv.org/pdf/1301.3781.pdf)并通过附加功能进行扩展。

### Imports and logging

In [1]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
接下来是我们的数据集。让Word2Vec真正为您工作的秘诀是拥有大量的文本数据。在这种情况下，我将使用[OpinRank](http://kavita-ganesan.com/entity-ranking-data/)的数据数据集。该数据集包含汽车和酒店的完整用户评论。我特别将所有酒店评论连接到一个大文件中，该文件约为97MB压缩和229MB未压缩。我们将在本教程中使用压缩文件。此文件中的每一行都代表一个酒店评论。您可以在此处下载OpinRank Word2Verc数据集。

为了避免混淆，虽然gensim的word2vec教程说你需要给它传递一系列句子作为输入，但你总是可以把整个评论作为一个句子（即大得多的文本）传递给它，这应该没有多大区别。

现在，让我们通过打印第一行来仔细查看下面的数据。你可以看到这是一个相当大的评论。

In [7]:

with open (path_to_file, 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b'First Citizen:\n'


### 将文件读取到列表中
现在我们已经有了数据集的峰值，我们可以将其读取到列表中，以便将其传递到Word2Verc模型中。请注意，在下面的代码中，我正在直接阅读
压缩文件。我还使用“gensim.utils.simple_proprocess（line）”对评论进行了轻度预处理。这会进行一些基本的预处理，如标记化、降低标记等，并返回一个标记（单词）列表。

此预处理方法的文档可在官方[Gensim文档网站](https://radimrehurek.com/gensim/utils.html)上找到.



In [10]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (path_to_file))
logging.info ("Done reading data file")    

2024-01-22 18:35:31,566 : INFO : reading file /Users/zhangchunyang/.keras/datasets/shakespeare.txt...this may take a while
2024-01-22 18:35:31,570 : INFO : read 0 reviews
2024-01-22 18:35:31,690 : INFO : read 10000 reviews
2024-01-22 18:35:31,787 : INFO : read 20000 reviews
2024-01-22 18:35:31,912 : INFO : read 30000 reviews
2024-01-22 18:35:32,002 : INFO : Done reading data file


## 训练Word2Verc模型

训练模型相当简单。您只需实例化Word2Vec并通过我们在上一步中阅读的评论（“文档”）。所以，我们本质上是在传递一个列表列表。其中，主列表中的每个列表都包含来自用户评论的一组令牌。Word2Vec使用所有这些标记在内部创建词汇表。我所说的词汇是指一组独特的单词。


我们实际上是在训练一个具有单个隐藏层的简单神经网络。但是，我们实际上不会在训练后使用神经网络。相反，目标是学习隐藏层的权重。这些权重本质上就是我们试图学习的单词向量。

In [13]:
model = gensim.models.Word2Vec(documents, vector_size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2024-01-22 18:37:05,241 : INFO : collecting all words and their counts
2024-01-22 18:37:05,243 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-01-22 18:37:05,258 : INFO : PROGRESS: at sentence #10000, processed 46762 words, keeping 5336 word types
2024-01-22 18:37:05,325 : INFO : PROGRESS: at sentence #20000, processed 99516 words, keeping 8036 word types
2024-01-22 18:37:05,341 : INFO : PROGRESS: at sentence #30000, processed 150456 words, keeping 9920 word types
2024-01-22 18:37:05,358 : INFO : collected 11442 word types from a corpus of 195977 raw words and 40000 sentences
2024-01-22 18:37:05,360 : INFO : Creating a fresh vocabulary
2024-01-22 18:37:05,395 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=2 retains 6526 unique words (57.04% of original 11442, drops 4916)', 'datetime': '2024-01-22T18:37:05.395176', 'gensim': '4.3.2', 'python': '3.8.2 (default, Dec  9 2020, 12:15:00) \n[Clang 12.0.0 (clang-1200.0.31.1)]', 'platform': 'macOS

(1467134, 1959770)

## 现在，让我们看看一些输出
第一个例子显示了查找与单词“dirty”类似的单词的简单情况。这里我们所需要做的就是调用“most_similate”函数，并提供单词“dirty”作为正面示例。这将返回前10个相似的单词。

In [16]:

w1 = "king"
model.wv.most_similar (positive=w1)


[('prince', 0.7226690053939819),
 ('crown', 0.717072069644928),
 ('bolingbroke', 0.717056930065155),
 ('right', 0.6728043556213379),
 ('deposed', 0.6683065295219421),
 ('warwick', 0.6600245833396912),
 ('succeed', 0.6251237392425537),
 ('devil', 0.6240665316581726),
 ('fowl', 0.6228092908859253),
 ('edward', 0.6213662028312683)]

看起来不错，对吧？让我们再看几个。让我们来看看“礼貌”、“法国”和“震惊”的相似之处。

In [18]:
# look up top 6 words similar to 'polite'
w1 = ["man"]
model.wv.most_similar (positive=w1,topn=6)


[('thing', 0.8096351623535156),
 ('nothing', 0.7957566380500793),
 ('woman', 0.77407306432724),
 ('born', 0.7342748045921326),
 ('mad', 0.7246645092964172),
 ('deed', 0.7215275764465332)]

In [19]:
# look up top 6 words similar to 'france'
w1 = ["apple"]
model.wv.most_similar (positive=w1,topn=6)


[('unstain', 0.822390615940094),
 ('mountain', 0.814261257648468),
 ('bauble', 0.8122409582138062),
 ('jove', 0.8109042048454285),
 ('whelp', 0.8053334355354309),
 ('trimm', 0.8050435781478882)]

In [20]:
# look up top 6 words similar to 'shocked'
w1 = ["sorrow"]
model.wv.most_similar (positive=w1,topn=6)


[('beauty', 0.8458828926086426),
 ('withholds', 0.8414062261581421),
 ('price', 0.8320472836494446),
 ('pride', 0.8306485414505005),
 ('strength', 0.8303678035736084),
 ('rankle', 0.8299378156661987)]

很好。您甚至可以指定几个积极的例子来获得在所提供的上下文中相关的东西，并提供消极的例子来说明不应该被视为相关的东西。在下面的例子中，我们只要求提供与床有关的所有项目：

In [21]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('shoulder', 0.9088001847267151),
 ('chamber', 0.899960994720459),
 ('boldness', 0.8979504108428955),
 ('maidenhead', 0.897590696811676),
 ('seal', 0.8911435604095459),
 ('drew', 0.8895896673202515),
 ('bosom', 0.8861256837844849),
 ('bidding', 0.8808514475822449),
 ('meditation', 0.8800074458122253),
 ('knee', 0.8786399960517883)]

### 词汇表中两个单词的相似性

您甚至可以使用Word2Verc模型来返回词汇表中两个单词之间的相似性。

In [26]:
# similarity between two different words
model.wv.similarity(w1="fine",w2="good")

0.23889905

在引擎盖下，以上三个片段使用每个单词的单词向量来计算两个指定单词之间的余弦相似度。从得分来看，“脏”与“臭”高度相似是有道理的，但“脏”不同于“干净”。如果你在两个相同的单词之间进行相似性测试，分数将为1.0，因为余弦相似性分数的范围将始终在[0.0-1.0]之间。你可以在[这里](https://en.wikipedia.org/wiki/Cosine_similarity)阅读更多关于余弦相似性评分的信息.

### 找出异常
您甚至可以使用Word2Vec来查找给定项目列表的异常项。

In [27]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

'france'

In [28]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


2024-01-22 18:40:08,047 : WARNING : vectors for words {'duvet'} are not present in the model, ignoring these words


'bed'

## 了解一些参数为了更早地训练模型，我们必须设置一些参数。

现在，让我们试着理解其中一些是什么意思。作为参考，这是我们用来训练模型的命令。
```
model=gensim.models.Word2Vec(documents，vecter_size=150，window=10，min_count=2，workers=10)
```
### `size`
表示每个标记或单词的密集向量的大小。如果您的数据非常有限，那么大小应该是一个小得多的值。如果你有很多数据，那么尝试各种尺寸是很好的。100-150的数值对我来说效果很好。

### `window`
目标单词与其相邻单词之间的最大距离。如果邻居的位置大于左右两侧的最大窗口宽度，则某些邻居不被视为与目标词相关。理论上，一个较小的窗口应该会给你提供更相关的术语。如果你有很多数据，那么窗口大小应该不太重要，只要它是一个大小合适的窗口。

### `min_count`
最小化单词的频率计数。该模型将忽略那些不能统计“min_count”的单词。极不常用的词通常是不重要的，所以最好去掉这些词。除非数据集非常小，否则这不会真正影响模型。

### `workers`
在幕后使用多少线程？
